In [1]:
import os
from dotenv import load_dotenv
# ✅ Step 1: Load environment variables
load_dotenv()
# Make sure to set your OpenAI API key in the .env file or directly here

# ✅ Step 2: Import libraries
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain import hub
import openai
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_community.retrievers import BM25Retriever
from langchain.docstore.document import Document
from langchain.retrievers import EnsembleRetriever
import asyncio
import nest_asyncio
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())
nest_asyncio.apply()
import gradio as gr

openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure you have set this in your .env file
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)
vectorstore = FAISS.load_local("nba_vector_db_semantic", embeddings=embedding, allow_dangerous_deserialization=True)

In [2]:
documents = vectorstore.docstore._dict.values()

In [40]:
## Define the Dense Retriever and BM25 Retriever
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
sparse_retriever = BM25Retriever.from_documents(list(documents), k=10)
ensemble_retriever = EnsembleRetriever(retrievers=[dense_retriever, sparse_retriever], weights=[0.5, 0.5], c=0)

In [41]:
## Define the prompt
prompt = hub.pull("jclemens24/rag-prompt")

c:\Users\16343\anaconda3\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [42]:
relevance_prompt_template = PromptTemplate.from_template(
    """
    Given the following question and retrieved context, determine if the context is relevant to the question.
    Provide a score from 1 to 5, where 1 is not at all relevant and 5 is highly relevant.
    Return ONLY the numeric score, without any additional text or explanation.

    Question: {question}
    Retrieved Context: {retrieved_context}

    Relevance Score:"""
)

In [43]:
def format_docs(docs):
 return "\n\n".join(doc.page_content for doc in docs)

In [44]:
def extract_score(llm_output):
    try:
        score = float(llm_output.strip())
        return score
    except ValueError:
        return 0

# Chain it all together with LangChain
def conditional_answer(x):
    relevance_score = extract_score(x['relevance_score'])
    if relevance_score < 2:
        return "I don't know."
    else:
        return x['answer']

In [45]:
## Define the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini")
str_output_parser = StrOutputParser()

In [46]:
## Define the chain
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | RunnableParallel(
        {"relevance_score": (
            RunnablePassthrough()
            | (lambda x: relevance_prompt_template.format(question=x['question'], retrieved_context=x['context']))
            | llm
            | str_output_parser
        ), "answer": (
            RunnablePassthrough()
            | prompt
            | llm
            | str_output_parser
        )}
    )
    | RunnablePassthrough().assign(final_answer=conditional_answer)
)

In [47]:
rag_chain_with_source = RunnableParallel(
    {"context": dense_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [50]:
result = rag_chain_with_source.invoke("Does Lebron James have a son?")
retrieved_docs = result['context']
relevance_score = result['answer']['relevance_score']
final_answer = result['answer']['final_answer']

print(f"Relevance Score: {relevance_score}")
print(f"Final Answer:\n{final_answer}")

Relevance Score: 5
Final Answer:
Yes, LeBron James has a son named Bronny James.


In [51]:
retrieved_docs

[Document(id='081825c2-cb83-47a0-afd7-932682509ba8', metadata={'player': 'Bronny James'}, page_content='LeBron Raymone "Bronny" James Jr. (born October 6, 2004) is an American professional basketball player for the Los Angeles Lakers of the National Basketball Association (NBA). A consensus four-star recruit, James was named a McDonald\'s All-American as a senior in high school in 2023. He played one season of college basketball for the USC Trojans before being selected by the Lakers in the second round of the 2024 NBA draft. He is the eldest child and teammate of professional basketball player LeBron James Sr., making them the first active father-son duo in NBA history. James was born on October 6, 2004, in Akron, Ohio, to reigning NBA Rookie of the Year LeBron James, age 19, and his girlfriend Savannah Brinson, age 18. He was raised by both of his parents, who married in 2013.'),
 Document(id='d9df9985-c83c-4924-b443-6b6cf06c4be1', metadata={'player': 'LeBron James'}, page_content='s

In [11]:
# Gradio Interface
def process_question(question):
    result = rag_chain_with_source.invoke(question)
    relevance_score = result['answer']['relevance_score']
    final_answer = result['answer']['final_answer']
    return relevance_score, final_answer

demo = gr.Interface(
    fn=process_question,
    inputs=gr.Textbox(label="Enter your question"),
    outputs=[
        gr.Textbox(label="Relevance Score"),
        gr.Textbox(label="Final Answer")
    ],
    title="NBA Player Question Answering",
    description="Enter a question and get the relevance score, final answer, and sources from RAG."
)

In [12]:
demo.launch(share=True, debug=False, auth=("admin", "pass1234")) # with credentials

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a4d65c183a651b0ab1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
